# Imports

In [2]:
import pandas as pd # básico para tratar con conjuntos de datos
import requests # realizar peticiones a las URLs
import tabula # trabajar los PDFS
import datetime # fechas
from datetime import timedelta

In [3]:
path = r"C:\Users\pcorral\Blog_Covid19\"

# Rango

In [ ]:
date_today = datetime.datetime.now()
var_date = datetime.datetime(2020, 3, 8) # fecha inicial donde empiezan los PDFs
days_to_loop = (date_today - var_date).days
start_loop = 39 # PDF inicial donde empezó a contabilizarse en el origen. Esto no cambiará
end_loop = start_loop + days_to_loop

rango_a_iterar = range(start_loop, end_loop)

# Descargar PDFs

In [4]:
# Bucle para guardar todos los PDFs en local
for w in rango_a_iterar: # iteramos sobre todos los PDFs
    name = f"covid_{str(w)}.pdf"
    # realizamos la petición a la URL para poder guardar el PDF
    doc = requests.get(f"https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_{str(w)}_COVID-19.pdf", verify=False)
    with open(path + name, 'wb') as f: 
        f.write(doc.content) # una vez leído el PDF, lo almacenamos en local

C:\Users\pcorral\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\pcorral\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\pcorral\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\pcorral\Ap

# Unificar conjunto de datos 

In [22]:
# FORMATOS
format_1 = ['CCAA', 'Total casos', 'IA (casos/100.000 habitantes)', 'Ingreso en UCI', 'Fallecidos']
format_1_reformat = ['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI','Fallecidos']

format_2 = ['CCAA', 'Total casos', 'IA Total*', 'Ingreso en UCI', 'Fallecidos']

format_3 = ['CCAA', 'Total casos', 'IA últimos 14 días', 'Ingreso en UCI','Fallecidos']

format_4 = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Nuevos']
format_4_reformat = ['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'Hospitalizados', 'UCI', 'Fallecidos', 'Nuevos']

format_5 = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Casos que han',
       'Casos que han.1', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
format_5_reformat = ['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos']

format_6 = ['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos']

ideal_format = format_6 # el formato ideal será el último (el más actual)

df = pd.DataFrame()

for i in rango_a_iterar: # iteramos sobre todos los PDFs
    df_aux = None
    var_date = var_date + timedelta(days=1)
    
    # añadimos un try-catch por si hubiera algun día intermedio donde no se han subido datos, que continúe con los siguientes
    # días. De hecho, existen dos días que, por alguna razón, no se han subido PDFs 
    try:
        # identificamos que la página donde se encuentra la tabla con los datos cambia a la primera 
        # cuando el nº del pdf es menor al 47
        if i < 47:
            df_aux = tabula.read_pdf(path + f"covid_{str(i)}.pdf", pages = "2", multiple_tables = True)
        else:
            df_aux = tabula.read_pdf(path + f"covid_{str(i)}.pdf", pages = "1", multiple_tables = True)
    except:
        continue
    
    # La tabla se puede encontrar o en la posición 0 o en la 1, dependiendo de como se haya construido la
    # tabla incial en el PDF
    if len(df_aux) == 1:
        df_aux = df_aux[0]
    else:
        df_aux = df_aux[1]

    cols_to_add = []

    # Reformateamos la tabla, añadiendo alguna columna si fuera necesario y cambiamos el nombre de las que si están
    # ya presentes pero se llaman diferente.
    if list(df_aux.columns) == format_1 or list(df_aux.columns) == format_2 or list(df_aux.columns) == format_3:
        df_aux.columns = format_1_reformat
        cols_to_add = set(ideal_format) - set(format_1_reformat)
        print(cols_to_add)
    elif list(df_aux.columns) == format_4:
        df_aux = df_aux[2:]
        df_aux.columns = format_4_reformat
        cols_to_add = set(ideal_format) - set(format_1_reformat)
    elif list(df_aux.columns) == format_3:
        df_aux = df_aux[2:]
        cols_to_add = set(ideal_format) - set(format_1_reformat)
    elif list(df_aux.columns) == format_5:
        df_aux = df_aux[5:]
        df_aux.columns = format_5_reformat
        cols_to_add = set(ideal_format) - set(format_5_reformat)

    # Ponemos a nulo aquellas columnas que no se encontraban originalmente en la tabla, pues no sabemos su valor
    for c in cols_to_add:
        df_aux[c] = None

    df_aux['Fecha'] = var_date # añadimos la columna fecha
    
    df = pd.concat([df, df_aux.reset_index(drop=True)], axis=0)

# Eliminamos los totales
df = df[df['CCAA'] != 'ESPAÑA']
df = df[df['CCAA'] != 'Total']

{'Nuevos', 'Hospitalizados', 'Curados'}
i:39, d:2020-03-10 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:40, d:2020-03-11 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:41, d:2020-03-12 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:42, d:2020-03-13 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:43, d:2020-03-14 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nu

Error from tabula-java:
Error: Error: End-of-File, expected line


Error from tabula-java:
Error: Error: End-of-File, expected line




{'Nuevos', 'Hospitalizados', 'Curados'}
i:46, d:2020-03-17 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:47, d:2020-03-18 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:48, d:2020-03-19 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:49, d:2020-03-20 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nuevos',
       'Hospitalizados', 'Curados', 'Fecha'],
      dtype='object')
{'Nuevos', 'Hospitalizados', 'Curados'}
i:50, d:2020-03-21 00:00:00, c: Index(['CCAA', 'TOTAL conf.', 'IA (14 d.)', 'UCI', 'Fallecidos', 'Nu

# CSV

In [23]:
df.to_csv('data.csv', sep='\t', index=False, encoding="latin1")